# Search-6-CSP-Fundamentals

**Navigation** : [<< Search-5 GeneticAlgorithms](Search-5-GeneticAlgorithms.ipynb) | [Index](../README.md) | [Search-7 CSP-Consistency >>](Search-7-CSP-Consistency.ipynb)

## Problemes de Satisfaction de Contraintes - Fondamentaux

Ce notebook introduit le formalisme des **CSP** (Constraint Satisfaction Problems) et les algorithmes de base pour les resoudre. Nous implementons tout de zero, puis decouvrons la bibliotheque `python-constraint` pour une modelisation rapide.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Formaliser** un probleme sous forme de CSP (variables, domaines, contraintes) (Bloom : comprendre)
2. **Implementer** l'algorithme de backtracking pour CSP (Bloom : appliquer)
3. **Appliquer** les heuristiques MRV et LCV pour accelerer la recherche (Bloom : analyser)
4. **Modeliser** des problemes classiques avec la bibliotheque `python-constraint` (Bloom : appliquer)
5. **Comparer** les approches manuelles et bibliotheque sur des problemes concrets (Bloom : evaluer)

### Prerequis
- Bases de Python (recursion, dictionnaires)
- Notions de recherche dans un espace d'etats (Search-1)

### Duree estimee : 50 minutes

### Lien avec d'autres series

Voir aussi la serie [Sudoku](../../Sudoku/README.md) pour une application complete des CSP.

---

## 1. Introduction (~5 min)

### Qu'est-ce qu'un CSP ?

Un **CSP** (Constraint Satisfaction Problem) est un probleme defini par un ensemble de variables qui doivent prendre des valeurs dans des domaines donnes, tout en respectant un ensemble de contraintes.

### CSP vs recherche classique

Dans les notebooks precedents (Search-1 a Search-3), nous avons explore la recherche dans un espace d'etats : BFS, DFS, A*. Ces methodes traitent le probleme comme un parcours de graphe. Les algorithmes genetiques (Search-5) traitent le probleme comme une optimisation.

Cependant, de nombreux problemes reels ont une structure particuliere que ces approches n'exploitent pas pleinement :

| Approche | Avantage | Limitation |
|----------|----------|------------|
| Recherche classique (BFS/DFS) | Generale | Ne tire pas parti de la structure du probleme |
| Recherche informee (A*) | Optimale avec bonne heuristique | Necessite une heuristique specifique |
| **CSP** | Les contraintes eliminent de larges portions de l'espace | Necessite une modelisation CSP |

### Exemples concrets de CSP

| Domaine | Probleme | Variables | Contraintes |
|---------|----------|-----------|-------------|
| Planification | Emplois du temps | Creneaux par cours | Pas de conflit salle/enseignant |
| Jeux | Sudoku | Cases de la grille | Chiffres distincts par ligne/colonne/bloc |
| Cartographie | Coloration de carte | Couleur par region | Regions adjacentes de couleurs differentes |
| Cryptographie | Cryptarithmetique | Lettres -> chiffres | L'equation arithmetique est respectee |

L'avantage fondamental des CSP est que les **contraintes** permettent de detecter les impasses **avant** d'avoir complete une solution, reduisant drastiquement l'exploration.

In [ ]:
# Imports pour tout le notebook
import sys
import copy
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Helpers partages de la serie Search
sys.path.insert(0, '..')
from search_helpers import draw_csp_graph

print("Imports OK")

---

## 2. Formalisation CSP (~8 min)

### Definition

Un **Probleme de Satisfaction de Contraintes** (CSP) est defini par un triplet $(X, D, C)$ :

- $X = \{X_1, X_2, \ldots, X_n\}$ : un ensemble de **variables**
- $D = \{D_1, D_2, \ldots, D_n\}$ : un ensemble de **domaines**, ou $D_i$ est l'ensemble des valeurs possibles pour $X_i$
- $C = \{C_1, C_2, \ldots, C_m\}$ : un ensemble de **contraintes**, chacune portant sur un sous-ensemble de variables

### Types de contraintes

| Type | Porte sur | Exemple |
|------|-----------|--------|
| **Unaire** | 1 variable | $X_1 \neq \text{Rouge}$ |
| **Binaire** | 2 variables | $X_1 \neq X_2$ |
| **Globale** | $k > 2$ variables | `AllDifferent(X_1, X_2, \ldots, X_k)` |

### Terminologie

| Terme | Definition |
|-------|------------|
| **Assignation** | Affectation d'une valeur a une ou plusieurs variables |
| **Assignation consistante** | Assignation qui ne viole aucune contrainte |
| **Assignation complete** | Toutes les variables ont une valeur |
| **Solution** | Assignation a la fois **complete** et **consistante** |

### Graphe de contraintes

On represente un CSP binaire par un **graphe de contraintes** ou :
- Chaque **noeud** est une variable
- Chaque **arete** relie deux variables partageant une contrainte

Implementons une classe CSP generique qui servira de base pour tout le notebook.

In [ ]:
class CSP:
    """Probleme de Satisfaction de Contraintes (CSP).

    Represente un CSP binaire avec variables, domaines, voisins
    et une fonction de contrainte.

    Attributs:
        variables: liste des noms de variables
        domains: dict variable -> liste de valeurs possibles
        neighbors: dict variable -> liste de variables voisines
        constraint_func: fonction (var1, val1, var2, val2) -> bool
        n_assigns: compteur d'assignations tentees
    """

    def __init__(self, variables, domains, neighbors, constraint_func):
        self.variables = variables
        self.domains = {v: list(d) for v, d in domains.items()}  # copie
        self.neighbors = neighbors
        self.constraint_func = constraint_func
        self.n_assigns = 0

    def consistent(self, var, val, assignment):
        """Verifie si (var=val) est consistant avec l'assignation partielle."""
        for other_var in self.neighbors[var]:
            if other_var in assignment:
                if not self.constraint_func(var, val, other_var, assignment[other_var]):
                    return False
        return True

    def is_complete(self, assignment):
        """Verifie si toutes les variables sont assignees."""
        return len(assignment) == len(self.variables)

    def is_solution(self, assignment):
        """Verifie si l'assignation est une solution (complete et consistante)."""
        if not self.is_complete(assignment):
            return False
        for var in self.variables:
            if not self.consistent(var, assignment[var], assignment):
                return False
        return True

    def reset_counter(self):
        """Reinitialise le compteur d'assignations."""
        self.n_assigns = 0

    def copy_domains(self):
        """Retourne une copie profonde des domaines."""
        return {v: list(d) for v, d in self.domains.items()}

    def get_constraints_list(self):
        """Retourne la liste des paires (var1, var2) de contraintes."""
        constraints = []
        seen = set()
        for var in self.variables:
            for neighbor in self.neighbors[var]:
                pair = tuple(sorted([var, neighbor]))
                if pair not in seen:
                    seen.add(pair)
                    constraints.append(pair)
        return constraints

print("Classe CSP definie.")

---

## 3. Exemple : coloration de carte de l'Australie (~8 min)

Le probleme classique de coloration de carte consiste a colorier les regions d'une carte de sorte que deux regions adjacentes n'aient jamais la meme couleur. C'est l'exemple canonique du livre AIMA (Russell & Norvig), applique a la carte de l'Australie avec ses 7 etats/territoires.

### Modelisation CSP

| Composant | Valeur |
|-----------|--------|
| **Variables** | WA, NT, SA, Q, NSW, V, T (les 7 etats australiens) |
| **Domaines** | {Rouge, Vert, Bleu} pour chaque variable |
| **Contraintes** | Regions adjacentes $\neq$ meme couleur |

Combien de combinaisons possibles (sans contraintes) ? $3^7 = 2187$. Les contraintes vont eliminer la grande majorite de ces combinaisons.

In [ ]:
# Definition du probleme de coloration de l'Australie
australia_vars = ['WA', 'NT', 'SA', 'Q', 'NSW', 'V', 'T']

australia_domains = {v: ['Rouge', 'Vert', 'Bleu'] for v in australia_vars}

australia_neighbors = {
    'WA':  ['NT', 'SA'],
    'NT':  ['WA', 'SA', 'Q'],
    'SA':  ['WA', 'NT', 'Q', 'NSW', 'V'],
    'Q':   ['NT', 'SA', 'NSW'],
    'NSW': ['Q', 'SA', 'V'],
    'V':   ['SA', 'NSW'],
    'T':   []  # La Tasmanie n'est adjacente a aucun etat continental
}

def different_values(var1, val1, var2, val2):
    """Contrainte : deux variables voisines doivent avoir des valeurs differentes."""
    return val1 != val2

australia_csp = CSP(australia_vars, australia_domains,
                    australia_neighbors, different_values)

# Affichage des informations du CSP
print("Probleme de coloration de l'Australie")
print("=" * 45)
print(f"Variables     : {australia_vars}")
print(f"Taille domaine: {len(australia_domains['WA'])} couleurs")
print(f"Contraintes   : {len(australia_csp.get_constraints_list())} paires")
print(f"\nEspace brut   : {3**7} combinaisons")
print(f"\nAdjacences :")
for v, n in australia_neighbors.items():
    print(f"  {v:>3} -> {n}")

### Visualisation du graphe de contraintes

Le graphe de contraintes montre les relations d'adjacence entre les etats. Chaque arete represente une contrainte binaire "les couleurs doivent etre differentes".

In [ ]:
# Visualisation avec le helper de la serie Search
constraints_list = australia_csp.get_constraints_list()
draw_csp_graph(
    australia_vars,
    australia_domains,
    constraints_list,
    title="Graphe de contraintes - Australie",
    figsize=(10, 7)
)
plt.show()

### Interpretation : graphe de contraintes

**Sortie obtenue** : le graphe montre les 7 etats australiens relies par 9 contraintes binaires.

| Variable | Degre (nb voisins) | Role dans le graphe |
|----------|--------------------|---------------------|
| SA | 5 | Noeud le plus contraint (centre du continent) |
| NT, Q, NSW | 3 | Degre intermediaire |
| WA, V | 2 | Peripherie |
| T | 0 | Isole (Tasmanie) |

**Points cles** :
1. SA est adjacent a presque tous les etats continentaux : c'est la variable la plus contrainte
2. T (Tasmanie) n'a aucune contrainte : n'importe quelle couleur convient
3. Le graphe n'est pas complet : seules les paires adjacentes sont contraintes

---

## 4. Backtracking Search (~10 min)

### Pourquoi pas la brute force ?

L'approche naive (generate-and-test) consiste a generer toutes les $3^7 = 2187$ combinaisons et filtrer celles qui satisfont les contraintes. C'est intraitable pour des problemes plus grands ($10^{50}$ pour 50 variables a 10 valeurs).

### Principe du backtracking

Le **backtracking** est l'algorithme de base pour resoudre un CSP :

1. Choisir une variable non encore assignee
2. Pour chaque valeur de son domaine :
   - Verifier la **consistance** avec l'assignation partielle
   - Si consistant, assigner et recurser sur les variables restantes
   - Sinon, essayer la valeur suivante
3. Si aucune valeur n'est valable, **revenir en arriere** (backtrack)

### Difference avec DFS classique

| Aspect | DFS classique | Backtracking CSP |
|--------|--------------|------------------|
| Assignation | Complete d'abord, verification ensuite | Verification **a chaque etape** |
| Elagage | Aucun | Elimination des branches inconsistantes |
| Profondeur | Variable | Exactement $n$ (nombre de variables) |

Commencons par mesurer le cout de la brute force, puis implementons le backtracking pour montrer l'amelioration.

In [ ]:
from itertools import product

# Brute force : enumerer toutes les combinaisons
colors = ['Rouge', 'Vert', 'Bleu']
n_total = 0
n_solutions = 0

start = time.time()
for combo in product(colors, repeat=7):
    n_total += 1
    assignment = dict(zip(australia_vars, combo))
    # Verifier toutes les contraintes
    valid = True
    for v1, v2 in constraints_list:
        if assignment[v1] == assignment[v2]:
            valid = False
            break
    if valid:
        n_solutions += 1
elapsed = time.time() - start

print("Brute force - Coloration Australie")
print("=" * 45)
print(f"Combinaisons testees  : {n_total}")
print(f"Solutions trouvees    : {n_solutions}")
print(f"Ratio solutions       : {n_solutions/n_total:.2%}")
print(f"Temps brute force     : {elapsed*1000:.1f} ms")

### Interpretation : brute force

**Sortie obtenue** : sur 2187 combinaisons possibles, seule une faible proportion satisfait toutes les contraintes.

| Mesure | Valeur | Signification |
|--------|--------|---------------|
| Espace total | 2187 | $3^7$ combinaisons |
| Solutions | ~18 | Depend de la symetrie des couleurs |
| Ratio | < 1% | L'immense majorite des combinaisons est invalide |

> **Conclusion** : il faut un algorithme qui exploite les contraintes pour **elaguer** l'espace de recherche. C'est le role du backtracking.

### Implementation du backtracking

In [ ]:
def backtracking_search(csp, assignment=None):
    """Backtracking simple pour CSP.

    Choisit les variables dans l'ordre de csp.variables.
    Explore les valeurs dans l'ordre du domaine.

    Retourne l'assignation solution ou None.
    """
    if assignment is None:
        assignment = {}

    # Cas de base : toutes les variables sont assignees
    if csp.is_complete(assignment):
        return assignment

    # Choisir la premiere variable non assignee (ordre naif)
    unassigned = [v for v in csp.variables if v not in assignment]
    var = unassigned[0]

    # Essayer chaque valeur du domaine
    for val in csp.domains[var]:
        csp.n_assigns += 1
        if csp.consistent(var, val, assignment):
            assignment[var] = val
            result = backtracking_search(csp, assignment)
            if result is not None:
                return result
            del assignment[var]

    return None  # Echec : backtrack

print("Fonction backtracking_search definie.")

Appliquons le backtracking au probleme de coloration de l'Australie et observons le nombre d'assignations necessaires.

In [ ]:
# Resolution par backtracking simple
csp_bt = CSP(australia_vars, australia_domains,
             australia_neighbors, different_values)

start = time.time()
solution = backtracking_search(csp_bt)
elapsed = time.time() - start

print("Backtracking simple - Coloration Australie")
print("=" * 45)
print(f"Solution : {solution}")
print(f"Assignations tentees : {csp_bt.n_assigns}")
print(f"Temps : {elapsed*1000:.2f} ms")

# Verification
if solution:
    print(f"\nVerification : {'VALIDE' if csp_bt.is_solution(solution) else 'INVALIDE'}")

### Interpretation : backtracking simple

**Sortie obtenue** : le backtracking trouve une solution avec beaucoup moins d'explorations que la brute force.

| Methode | Essais | Amelioration |
|---------|--------|-------------|
| Brute force | 2187 | reference |
| Backtracking | ~7-15 | environ 200x moins |

**Pourquoi cette reduction ?** Le backtracking detecte les inconsistances des qu'elles apparaissent. Si WA = Rouge et NT = Rouge, la contrainte `WA != NT` est violee immediatement : toutes les extensions de cette assignation partielle sont eliminees sans etre explorees.

> **Question** : peut-on faire encore mieux en choisissant plus intelligemment quelle variable assigner en premier et quelle valeur essayer ?

### Trace detaillee du backtracking

Pour mieux comprendre le fonctionnement, implementons une version avec trace qui montre chaque decision et chaque retour en arriere.

In [ ]:
def backtracking_search_verbose(csp, assignment=None, depth=0):
    """Backtracking avec trace detaillee des decisions."""
    if assignment is None:
        assignment = {}

    indent = "  " * depth

    if csp.is_complete(assignment):
        print(f"{indent}>> Solution trouvee !")
        return assignment

    unassigned = [v for v in csp.variables if v not in assignment]
    var = unassigned[0]

    for val in csp.domains[var]:
        csp.n_assigns += 1
        if csp.consistent(var, val, assignment):
            print(f"{indent}{var} = {val}  (consistant)")
            assignment[var] = val
            result = backtracking_search_verbose(csp, assignment, depth + 1)
            if result is not None:
                return result
            del assignment[var]
            print(f"{indent}{var} = {val}  -> backtrack")
        else:
            print(f"{indent}{var} = {val}  (CONFLIT)")

    return None

# Trace sur un sous-probleme plus petit (4 regions)
small_vars = ['A', 'B', 'C', 'D']
small_domains = {v: ['R', 'V', 'B'] for v in small_vars}
small_neighbors = {
    'A': ['B', 'C'],
    'B': ['A', 'C'],
    'C': ['A', 'B', 'D'],
    'D': ['C']
}
small_csp = CSP(small_vars, small_domains, small_neighbors, different_values)

print("Trace du backtracking sur un graphe a 4 noeuds")
print("=" * 50)
sol = backtracking_search_verbose(small_csp)
print(f"\nSolution : {sol}")
print(f"Assignations tentees : {small_csp.n_assigns}")

### Interpretation : trace du backtracking

La trace montre le processus de decision pas a pas :

1. L'algorithme assigne une valeur a la premiere variable
2. Il detecte les conflits immediatement grace a la verification de consistance
3. Quand un conflit survient sur toutes les valeurs, il revient en arriere (backtrack)

**Observations** :
- Les conflits sont detectes **des l'assignation**, pas a la fin
- Le "backtrack" est visible dans la trace par les retours en arriere
- L'indentation montre la profondeur de recursion

---

## 5. Heuristiques : MRV et LCV (~8 min)

Le backtracking simple fonctionne, mais deux choix strategiques peuvent considerablement l'accelerer :

1. **Quelle variable** assigner ensuite ? (variable ordering)
2. **Quelle valeur** essayer en premier ? (value ordering)

### 5.1 Variable Ordering : MRV (Minimum Remaining Values)

**Principe** : choisir la variable qui a le **plus petit domaine restant** (le moins de valeurs viables).

**Intuition ("fail-first")** : si une variable n'a que 2 valeurs possibles et une autre en a 10, mieux vaut traiter d'abord celle a 2 valeurs. Si elle echoue, on le decouvre plus vite, et on elague un sous-arbre plus tot.

$$\text{MRV}(X_i) = |\{v \in D_i : v \text{ est consistant avec l'assignation courante}\}|$$

On choisit $X_i$ qui minimise $\text{MRV}(X_i)$.

In [ ]:
def select_mrv(csp, assignment):
    """Heuristique MRV : choisir la variable avec le moins de valeurs viables.

    En cas d'egalite, departage par degree heuristic (nombre de voisins
    non assignes, en ordre decroissant).
    """
    unassigned = [v for v in csp.variables if v not in assignment]

    def remaining_values(var):
        return sum(1 for val in csp.domains[var]
                   if csp.consistent(var, val, assignment))

    def degree(var):
        return sum(1 for n in csp.neighbors[var] if n not in assignment)

    # MRV croissant, puis degree decroissant en cas d'egalite
    return min(unassigned, key=lambda v: (remaining_values(v), -degree(v)))

print("Heuristique MRV definie.")

Illustrons le choix MRV : apres quelques assignations, voyons quelle variable est selectionnee et pourquoi.

In [ ]:
# Demonstration du choix MRV
demo_csp = CSP(australia_vars, australia_domains,
               australia_neighbors, different_values)
partial = {'WA': 'Rouge', 'NT': 'Vert'}  # assignation partielle

print("Assignation partielle : WA=Rouge, NT=Vert")
print("\nValeurs restantes viables pour chaque variable non assignee :")
print(f"{'Variable':<8} {'Valeurs viables':<30} {'MRV':>5} {'Degree':>8}")
print("-" * 55)

for var in demo_csp.variables:
    if var not in partial:
        viable = [val for val in demo_csp.domains[var]
                  if demo_csp.consistent(var, val, partial)]
        deg = sum(1 for n in demo_csp.neighbors[var] if n not in partial)
        print(f"{var:<8} {str(viable):<30} {len(viable):>5} {deg:>8}")

chosen = select_mrv(demo_csp, partial)
print(f"\n=> MRV selectionne : {chosen} (domaine le plus restreint, puis degre le plus eleve)")

### Interpretation : selection MRV

**Sortie obtenue** : apres avoir assigne WA=Rouge et NT=Vert, SA n'a plus qu'une seule valeur viable (Bleu), car elle est adjacente aux deux regions deja coloriees.

| Variable | Valeurs viables | Pourquoi |
|----------|----------------|----------|
| SA | [Bleu] seulement | Adjacent a WA (Rouge) et NT (Vert) |
| Q, NSW | 2 valeurs | Adjacents a certaines variables assignees |
| V, T | 3 valeurs | Pas ou peu de voisins assignes |

MRV choisit SA car c'est la variable la plus contrainte. Si SA n'a aucune valeur viable, on detecte l'echec immediatement sans explorer les autres variables.

### 5.2 Value Ordering : LCV (Least Constraining Value)

**Principe** : pour la variable choisie, essayer d'abord la valeur qui **elimine le moins de possibilites** pour les variables voisines non assignees.

**Intuition ("succeed-first")** : contrairement au "fail-first" pour les variables, pour les valeurs on prefere maximiser les chances que le reste du probleme reste soluble.

$$\text{LCV}(v) = \sum_{\text{voisin } Y \text{ non assigne}} |\{w \in D_Y : \text{contrainte}(X, v, Y, w) \text{ violee}\}|$$

On trie les valeurs par nombre de conflits croissant (la moins contraignante d'abord).

In [ ]:
def order_lcv(csp, var, assignment):
    """Heuristique LCV : trier les valeurs par nombre de conflits croissant."""
    def conflicts(val):
        count = 0
        for neighbor in csp.neighbors[var]:
            if neighbor not in assignment:
                for nval in csp.domains[neighbor]:
                    if not csp.constraint_func(var, val, neighbor, nval):
                        count += 1
        return count

    return sorted(csp.domains[var], key=conflicts)

print("Heuristique LCV definie.")

### Backtracking ameliore : MRV + LCV

Combinons les deux heuristiques dans une version amelioree du backtracking.

In [ ]:
def backtracking_improved(csp, assignment=None,
                          use_mrv=True, use_lcv=True):
    """Backtracking avec heuristiques MRV et LCV."""
    if assignment is None:
        assignment = {}

    if csp.is_complete(assignment):
        return assignment

    # Selection de variable
    if use_mrv:
        var = select_mrv(csp, assignment)
    else:
        unassigned = [v for v in csp.variables if v not in assignment]
        var = unassigned[0]

    # Ordre des valeurs
    if use_lcv:
        values = order_lcv(csp, var, assignment)
    else:
        values = csp.domains[var]

    for val in values:
        csp.n_assigns += 1
        if csp.consistent(var, val, assignment):
            assignment[var] = val
            result = backtracking_improved(csp, assignment, use_mrv, use_lcv)
            if result is not None:
                return result
            del assignment[var]

    return None

print("Backtracking ameliore defini.")

### Benchmark : plain backtracking vs MRV vs MRV+LCV

Comparons les differentes variantes sur le probleme de coloration australienne.

In [ ]:
# Comparaison des variantes sur la coloration de l'Australie
variants = [
    ("Backtracking simple", False, False),
    ("+ MRV",              True,  False),
    ("+ MRV + LCV",        True,  True),
]

results_australia = []

print("Comparaison des variantes - Coloration Australie")
print("=" * 55)
print(f"{'Variante':<25} {'Assignations':>12} {'Temps (ms)':>12}")
print("-" * 55)

for name, mrv, lcv in variants:
    csp = CSP(australia_vars, australia_domains,
              australia_neighbors, different_values)
    start = time.time()
    sol = backtracking_improved(csp, use_mrv=mrv, use_lcv=lcv)
    elapsed = (time.time() - start) * 1000

    results_australia.append({
        'name': name,
        'assigns': csp.n_assigns,
        'time_ms': elapsed,
        'solution': sol
    })
    print(f"{name:<25} {csp.n_assigns:>12} {elapsed:>12.2f}")

print(f"\nToutes les variantes trouvent une solution valide.")

### Interpretation : impact des heuristiques sur la coloration

**Sortie obtenue** : les heuristiques reduisent le nombre d'assignations.

| Variante | Assignations | Commentaire |
|----------|-------------|-------------|
| Backtracking simple | ~7-15 | Ordre naif des variables et valeurs |
| + MRV | ~7-10 | Fail-first sur les variables |
| + MRV + LCV | ~7 | Optimal ou quasi-optimal |

**Points cles** :
1. Sur ce petit probleme (7 variables, 3 couleurs), l'amelioration est modeste car l'espace est deja restreint
2. L'impact des heuristiques devient **dramatique** sur des problemes plus grands
3. MRV est generalement l'heuristique la plus importante ; LCV apporte un gain supplementaire

> **Regle pratique** : MRV seul offre souvent le meilleur rapport cout/benefice, car le calcul de LCV a un overhead non negligeable.

Visualisons l'impact des heuristiques sur les deux problemes : coloration et N-Reines.

In [ ]:
# N-Reines : definition du probleme
def make_nqueens_csp(n):
    """Cree un CSP pour le probleme des N-Reines."""
    variables = list(range(n))  # colonnes 0..n-1
    domains = {col: list(range(n)) for col in variables}
    neighbors = {col: [c for c in variables if c != col] for col in variables}

    def queens_constraint(c1, r1, c2, r2):
        if r1 == r2:
            return False  # meme ligne
        if abs(c1 - c2) == abs(r1 - r2):
            return False  # meme diagonale
        return True

    return CSP(variables, domains, neighbors, queens_constraint)


def draw_queens(solution, n, title="Solution N-Reines"):
    """Visualise la solution du probleme des N-Reines."""
    fig, ax = plt.subplots(figsize=(max(6, n * 0.8), max(6, n * 0.8)))

    for row in range(n):
        for col in range(n):
            color = '#F0D9B5' if (row + col) % 2 == 0 else '#B58863'
            rect = plt.Rectangle((col, n - 1 - row), 1, 1,
                                 facecolor=color, edgecolor='black')
            ax.add_patch(rect)

    if solution:
        for col, row in solution.items():
            ax.text(col + 0.5, n - 1 - row + 0.5, 'Q',
                    ha='center', va='center', fontsize=max(8, 24 - n),
                    fontweight='bold', color='darkred')

    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_aspect('equal')
    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels(range(n))
    ax.set_yticklabels(range(n - 1, -1, -1))
    ax.set_xlabel('Colonne')
    ax.set_ylabel('Ligne')
    ax.set_title(title, fontsize=13, fontweight='bold')
    plt.tight_layout()
    return fig

print("Fonctions N-Reines definies.")

Resolvons le probleme des 8-Reines et visualisons le resultat, puis comparons les variantes.

In [ ]:
# Resolution et visualisation des 8-Reines
n = 8
csp_queens = make_nqueens_csp(n)

start = time.time()
sol_queens = backtracking_improved(csp_queens, use_mrv=True, use_lcv=True)
elapsed = (time.time() - start) * 1000

print(f"Probleme des {n}-Reines")
print("=" * 35)
print(f"Solution (col -> ligne) : {sol_queens}")
print(f"Assignations tentees   : {csp_queens.n_assigns}")
print(f"Temps                  : {elapsed:.2f} ms")
print(f"Verification           : {'VALIDE' if csp_queens.is_solution(sol_queens) else 'INVALIDE'}")

draw_queens(sol_queens, n, f"Solution des {n}-Reines ({csp_queens.n_assigns} assignations)")
plt.show()

### Interpretation : 8-Reines

**Sortie obtenue** : le backtracking avec MRV trouve une solution au probleme des 8-Reines.

| Mesure | Valeur |
|--------|--------|
| Espace brut | $8^8 = 16\,777\,216$ |
| Assignations (avec MRV) | typiquement < 50 |
| Reduction | facteur > 300 000 |

**Points cles** :
1. L'echiquier montre qu'aucune reine n'est sur la meme ligne, colonne ou diagonale
2. La modelisation CSP (une variable par colonne) est plus efficace qu'une recherche dans l'espace complet
3. MRV est particulierement efficace ici car les domaines se reduisent vite a mesure que l'on place des reines

In [ ]:
# Comparaison des variantes sur les deux problemes
def benchmark_csp(make_csp_func, csp_name, csp_args=None):
    """Benchmark des variantes de backtracking sur un CSP."""
    configs = [
        ("Plain backtracking", False, False),
        ("+ MRV",             True,  False),
        ("+ MRV + LCV",       True,  True),
    ]
    results = []
    for name, mrv, lcv in configs:
        csp = make_csp_func() if csp_args is None else make_csp_func(*csp_args)
        start = time.time()
        sol = backtracking_improved(csp, use_mrv=mrv, use_lcv=lcv)
        elapsed = (time.time() - start) * 1000
        results.append({
            'name': name,
            'assigns': csp.n_assigns,
            'time_ms': elapsed,
            'found': sol is not None
        })
    return results

def make_australia():
    return CSP(australia_vars, australia_domains,
              australia_neighbors, different_values)

res_map = benchmark_csp(make_australia, "Coloration Australie")
res_queens = benchmark_csp(make_nqueens_csp, "8-Reines", csp_args=(8,))

# Tableau comparatif
print("Tableau comparatif des variantes")
print("=" * 70)
print(f"{'Variante':<22} {'Map (assigns)':>14} {'8-Reines (assigns)':>20}")
print("-" * 70)
for rm, rq in zip(res_map, res_queens):
    print(f"{rm['name']:<22} {rm['assigns']:>14} {rq['assigns']:>20}")
print("=" * 70)

Representons graphiquement les resultats du benchmark pour mieux visualiser l'ecart entre les variantes.

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

names = [r['name'] for r in res_map]
assigns_map = [r['assigns'] for r in res_map]
axes[0].barh(names, assigns_map, color=['#2196F3', '#4CAF50', '#FF9800'])
axes[0].set_xlabel('Assignations tentees')
axes[0].set_title('Coloration Australie', fontweight='bold')
for i, v in enumerate(assigns_map):
    axes[0].text(v + 0.1, i, str(v), va='center', fontsize=10)

assigns_q = [r['assigns'] for r in res_queens]
axes[1].barh(names, assigns_q, color=['#2196F3', '#4CAF50', '#FF9800'])
axes[1].set_xlabel('Assignations tentees')
axes[1].set_title('8-Reines', fontweight='bold')
for i, v in enumerate(assigns_q):
    axes[1].text(v + 0.1, i, str(v), va='center', fontsize=10)

plt.suptitle('Impact des heuristiques sur le backtracking CSP',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Interpretation : comparaison des variantes

| Variante | Coloration (assigns) | 8-Reines (assigns) | Commentaire |
|----------|---------------------|---------------------|-------------|
| Plain backtracking | reference | reference | Ordre naif des variables et valeurs |
| + MRV | reduction moderee | reduction significative | Fail-first sur les variables |
| + MRV + LCV | optimal ou quasi | optimal ou quasi | Succeed-first sur les valeurs |

**Observations** :
1. **MRV** est l'heuristique la plus impactante, surtout pour N-Reines ou les domaines se reduisent asymetriquement
2. **LCV** ajoute un gain supplementaire mais son cout de calcul peut compenser sur les petits problemes
3. Sur des problemes plus grands, l'ecart peut etre de plusieurs ordres de grandeur

> **A retenir** : les heuristiques de selection ne changent pas la correction du backtracking, mais peuvent transformer un probleme intraitable en probleme resolvable en temps raisonnable.

---

## 6. Bibliotheque python-constraint (~8 min)

Jusqu'ici nous avons tout implemente de zero, ce qui est essentiel pour comprendre les mecanismes. En pratique, on utilise souvent une **bibliotheque CSP** qui fournit un solveur optimise et des contraintes predefinies.

La bibliotheque [`python-constraint`](https://github.com/python-constraint/python-constraint) permet de modeliser et resoudre des CSP de maniere declarative.

### Installation

```bash
pip install python-constraint
```

In [ ]:
# Installation si necessaire
try:
    from constraint import Problem, AllDifferentConstraint
    print("python-constraint importe avec succes.")
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-constraint'])
    from constraint import Problem, AllDifferentConstraint
    print("python-constraint installe et importe.")

### Coloration de l'Australie avec python-constraint

Reecrivons le probleme de coloration de l'Australie avec la bibliotheque. Comparons la concision du code avec notre implementation manuelle.

In [ ]:
from constraint import Problem, AllDifferentConstraint

# Coloration Australie avec python-constraint
problem = Problem()

# Variables et domaines
colors = ['Rouge', 'Vert', 'Bleu']
problem.addVariables(['WA', 'NT', 'SA', 'Q', 'NSW', 'V', 'T'], colors)

# Contraintes : regions adjacentes differentes
adjacencies = [
    ('WA', 'NT'), ('WA', 'SA'), ('NT', 'SA'), ('NT', 'Q'),
    ('SA', 'Q'), ('SA', 'NSW'), ('SA', 'V'),
    ('Q', 'NSW'), ('NSW', 'V')
]
for v1, v2 in adjacencies:
    problem.addConstraint(lambda a, b: a != b, (v1, v2))

# Trouver une solution
start = time.time()
sol_lib = problem.getSolution()
elapsed = (time.time() - start) * 1000

print("python-constraint - Coloration Australie")
print("=" * 45)
print(f"Solution : {sol_lib}")
print(f"Temps    : {elapsed:.2f} ms")

# Trouver TOUTES les solutions
start = time.time()
all_solutions = problem.getSolutions()
elapsed_all = (time.time() - start) * 1000

print(f"\nNombre total de solutions : {len(all_solutions)}")
print(f"Temps (toutes solutions)  : {elapsed_all:.2f} ms")

### Interpretation : python-constraint vs implementation manuelle

**Sortie obtenue** : la bibliotheque trouve rapidement une solution et peut enumerer toutes les solutions.

| Aspect | Implementation manuelle | python-constraint |
|--------|------------------------|-------------------|
| Lignes de code | ~60 (classe CSP + backtracking) | ~10 |
| Toutes les solutions | Necessiterait une modification | `getSolutions()` integre |
| Contraintes predefinies | A coder soi-meme | `AllDifferentConstraint`, etc. |
| Pedagogie | Comprendre les mecanismes | Utiliser en production |

**Points cles** :
1. La bibliotheque abstrait les details d'implementation (backtracking, heuristiques)
2. `getSolutions()` enumere exhaustivement toutes les solutions valides
3. L'API declarative est plus lisible et moins sujette aux erreurs

### AllDifferentConstraint

La contrainte globale `AllDifferentConstraint` impose que toutes les variables d'un groupe prennent des valeurs distinctes. C'est la contrainte la plus frequente en pratique (Sudoku, emplois du temps, N-Reines).

In [ ]:
# Demonstration AllDifferentConstraint
# 4 variables devant toutes etre differentes, domaine {1, 2, 3, 4}
p = Problem()
p.addVariables(['A', 'B', 'C', 'D'], [1, 2, 3, 4])
p.addConstraint(AllDifferentConstraint())

solutions = p.getSolutions()
print(f"AllDifferent sur 4 variables, domaine {{1,2,3,4}}")
print(f"Nombre de solutions : {len(solutions)}  (= 4! = {4*3*2*1})")
print(f"\nExemples (3 premieres) :")
for s in solutions[:3]:
    print(f"  {s}")

### 4-Reines avec python-constraint

Modelisons le probleme des 4-Reines avec la bibliotheque pour montrer sa simplicite.

In [ ]:
# 4-Reines avec python-constraint
n = 4
queens_problem = Problem()

# Variables : Q0, Q1, Q2, Q3 (colonne -> ligne)
queens_problem.addVariables(range(n), range(n))

# Contraintes : pas meme ligne, pas meme diagonale
for i in range(n):
    for j in range(i + 1, n):
        # Pas meme ligne et pas meme diagonale
        queens_problem.addConstraint(
            lambda qi, qj, d=j-i: qi != qj and abs(qi - qj) != d,
            (i, j)
        )

# Toutes les solutions
start = time.time()
queens_solutions = queens_problem.getSolutions()
elapsed = (time.time() - start) * 1000

print(f"Probleme des {n}-Reines (python-constraint)")
print("=" * 45)
print(f"Nombre de solutions : {len(queens_solutions)}")
print(f"Temps               : {elapsed:.2f} ms")
print(f"\nSolutions :")
for i, sol in enumerate(queens_solutions):
    placement = [sol[col] for col in range(n)]
    print(f"  Solution {i+1} : lignes = {placement}")

# Visualiser les 2 solutions
fig, axes = plt.subplots(1, len(queens_solutions), figsize=(5 * len(queens_solutions), 5))
if len(queens_solutions) == 1:
    axes = [axes]
for idx, sol in enumerate(queens_solutions):
    ax = axes[idx]
    for row in range(n):
        for col in range(n):
            color = '#F0D9B5' if (row + col) % 2 == 0 else '#B58863'
            rect = plt.Rectangle((col, n - 1 - row), 1, 1,
                                 facecolor=color, edgecolor='black')
            ax.add_patch(rect)
    for col in range(n):
        row = sol[col]
        ax.text(col + 0.5, n - 1 - row + 0.5, 'Q',
                ha='center', va='center', fontsize=18,
                fontweight='bold', color='darkred')
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_aspect('equal')
    ax.set_title(f"Solution {idx+1}", fontweight='bold')
    ax.axis('off')

plt.suptitle(f"Les {len(queens_solutions)} solutions du probleme des {n}-Reines",
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Interpretation : 4-Reines avec la bibliotheque

**Sortie obtenue** : le probleme des 4-Reines a exactement 2 solutions (symetriques l'une de l'autre).

| Aspect | Valeur |
|--------|--------|
| Solutions | 2 (symetrie miroir) |
| Espace brut | $4^4 = 256$ |
| Code | ~10 lignes vs ~50 en manuel |

**Points cles** :
1. La bibliotheque gere automatiquement le backtracking et les heuristiques
2. `getSolutions()` retourne la liste exhaustive des solutions
3. En production, on prefere utiliser une bibliotheque pour gagner en fiabilite et en temps de developpement

> **Quand utiliser quoi ?** Implementation manuelle pour **apprendre** les mecanismes. Bibliotheque pour **resoudre** des problemes reels.

---

## 7. Recapitulatif et exercices

### Tableau recapitulatif

| Approche | Noeuds explores | Temps | Facilite d'utilisation |
|----------|----------------|-------|------------------------|
| Brute force | $\prod |D_i|$ (tous) | Lent | Simple mais intraitable |
| Backtracking simple | Reduit (elagage) | Rapide | Implementation moderee |
| Backtracking + MRV | Tres reduit | Tres rapide | Implementation moderee |
| Backtracking + MRV + LCV | Minimal | Tres rapide | Plus complexe a implementer |
| python-constraint | Optimise (interne) | Rapide | Tres simple (declaratif) |

### Resume du formalisme CSP

| Concept | Definition | Exemple |
|---------|------------|--------|
| **Variable** ($X_i$) | Quantite a determiner | Region a colorier |
| **Domaine** ($D_i$) | Valeurs possibles pour $X_i$ | {Rouge, Vert, Bleu} |
| **Contrainte** ($C_k$) | Restriction sur un sous-ensemble de variables | $X_1 \neq X_2$ |
| **Solution** | Assignation complete et consistante | Toutes les regions coloriees sans conflit |

### Resume des algorithmes

| Algorithme / Heuristique | Principe | Impact |
|--------------------------|----------|--------|
| **Backtracking** | DFS avec verification de consistance a chaque etape | Base de toute resolution CSP |
| **MRV** | Choisir la variable au domaine le plus restreint | Fail-first : detecte les impasses tot |
| **LCV** | Essayer la valeur la moins contraignante | Succeed-first : maximise les chances |
| **python-constraint** | API declarative avec solveur integre | Productivite en production |

### Exercice 1 : 4-Reines avec backtracking manuel

**Enonce** : modelisez et resolvez le probleme des 4-Reines avec notre backtracking `backtracking_improved` (MRV + LCV). Affichez la solution avec `draw_queens`.

Completez la cellule ci-dessous.

In [ ]:
# Exercice 1 : 4-Reines avec backtracking manuel

# A COMPLETER
# csp_4q = make_nqueens_csp(4)
# sol_4q = backtracking_improved(csp_4q, use_mrv=True, use_lcv=True)
# print(f"Solution : {sol_4q}")
# print(f"Assignations : {csp_4q.n_assigns}")
# draw_queens(sol_4q, 4, "4-Reines - Backtracking manuel")
# plt.show()

<details>
<summary><b>Solution exercice 1</b></summary>

```python
csp_4q = make_nqueens_csp(4)
sol_4q = backtracking_improved(csp_4q, use_mrv=True, use_lcv=True)
print(f"Solution : {sol_4q}")
print(f"Assignations : {csp_4q.n_assigns}")
draw_queens(sol_4q, 4, "4-Reines - Backtracking manuel")
plt.show()
```

</details>

### Exercice 2 : comparer MRV sur les 4-Reines

**Enonce** : resolvez les 4-Reines **sans** MRV (`use_mrv=False`) et **avec** MRV. Comparez le nombre d'assignations. Le gain est-il significatif sur ce petit probleme ?

In [ ]:
# Exercice 2 : comparaison avec et sans MRV sur 4-Reines

# A COMPLETER
# csp_no_mrv = make_nqueens_csp(4)
# sol1 = backtracking_improved(csp_no_mrv, use_mrv=False, use_lcv=False)
# print(f"Sans MRV : {csp_no_mrv.n_assigns} assignations")
#
# csp_with_mrv = make_nqueens_csp(4)
# sol2 = backtracking_improved(csp_with_mrv, use_mrv=True, use_lcv=True)
# print(f"Avec MRV : {csp_with_mrv.n_assigns} assignations")
#
# if csp_with_mrv.n_assigns > 0:
#     print(f"Facteur  : {csp_no_mrv.n_assigns / csp_with_mrv.n_assigns:.1f}x")

<details>
<summary><b>Solution exercice 2</b></summary>

```python
csp_no_mrv = make_nqueens_csp(4)
sol1 = backtracking_improved(csp_no_mrv, use_mrv=False, use_lcv=False)
print(f"Sans MRV : {csp_no_mrv.n_assigns} assignations")

csp_with_mrv = make_nqueens_csp(4)
sol2 = backtracking_improved(csp_with_mrv, use_mrv=True, use_lcv=True)
print(f"Avec MRV : {csp_with_mrv.n_assigns} assignations")

if csp_with_mrv.n_assigns > 0:
    print(f"Facteur  : {csp_no_mrv.n_assigns / csp_with_mrv.n_assigns:.1f}x")
```

Le gain est modeste sur un si petit probleme. Essayez avec N=12 pour observer une difference significative.

</details>

### Exercice 3 : Cryptarithmetique SEND + MORE = MONEY

**Enonce** : le puzzle cryptarithmetique **SEND + MORE = MONEY** consiste a trouver l'affectation de chiffres (0-9) aux lettres telle que l'addition soit correcte. Chaque lettre represente un chiffre distinct, et S et M ne peuvent pas etre 0 (pas de zero initial).

```
    S E N D
  + M O R E
  ---------
  M O N E Y
```

Modelisez et resolvez ce probleme avec `python-constraint`.

**Indice** : la contrainte principale est :
$$1000 \times S + 100 \times E + 10 \times N + D + 1000 \times M + 100 \times O + 10 \times R + E$$
$$= 10000 \times M + 1000 \times O + 100 \times N + 10 \times E + Y$$

In [ ]:
# Exercice 3 : SEND + MORE = MONEY

# A COMPLETER
# from constraint import Problem, AllDifferentConstraint
#
# money = Problem()
# money.addVariables(???, range(0, 10))
# money.addConstraint(AllDifferentConstraint())
# # S et M != 0
# # Contrainte arithmetique
# sol = money.getSolution()
# print(f"Solution : {sol}")

<details>
<summary><b>Solution exercice 3</b></summary>

```python
from constraint import Problem, AllDifferentConstraint

money = Problem()
letters = ['S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y']
money.addVariables(letters, range(0, 10))
money.addConstraint(AllDifferentConstraint())

# S et M ne peuvent pas etre 0 (pas de zero initial)
money.addConstraint(lambda s: s != 0, ('S',))
money.addConstraint(lambda m: m != 0, ('M',))

# Contrainte arithmetique SEND + MORE = MONEY
def send_more_money(s, e, n, d, m, o, r, y):
    send  = 1000*s + 100*e + 10*n + d
    more  = 1000*m + 100*o + 10*r + e
    money_val = 10000*m + 1000*o + 100*n + 10*e + y
    return send + more == money_val

money.addConstraint(send_more_money, letters)

solutions = money.getSolutions()
print(f"Nombre de solutions : {len(solutions)}")
for sol in solutions:
    send  = 1000*sol['S'] + 100*sol['E'] + 10*sol['N'] + sol['D']
    more  = 1000*sol['M'] + 100*sol['O'] + 10*sol['R'] + sol['E']
    money_val = 10000*sol['M'] + 1000*sol['O'] + 100*sol['N'] + 10*sol['E'] + sol['Y']
    print(f"  {send} + {more} = {money_val}")
    print(f"  Assignation : {sol}")
```

La solution unique est : S=9, E=5, N=6, D=7, M=1, O=0, R=8, Y=2 (soit 9567 + 1085 = 10652).

</details>

### Et ensuite ?

Ce notebook a couvert les **fondamentaux** des CSP : formalisme, backtracking avec heuristiques, et utilisation de la bibliotheque `python-constraint`. Le notebook suivant, [Search-7 CSP-Consistency](Search-7-CSP-Consistency.ipynb), introduit les techniques de **propagation de contraintes** :

- **Forward Checking** : eliminer les valeurs inconsistantes des voisins a chaque assignation
- **Arc Consistency (AC-3)** : rendre le CSP arc-consistent avant et pendant la recherche
- **MAC (Maintaining Arc Consistency)** : combiner AC-3 avec le backtracking

Ces techniques permettent de reduire encore davantage l'exploration en detectant les impasses plus tot.

### References

- Russell, S. & Norvig, P. *Artificial Intelligence: A Modern Approach*, Chapitre 6
- Dechter, R. *Constraint Processing*, Cambridge University Press, 2003
- Voir aussi la serie [Sudoku](../../Sudoku/README.md) pour une application complete des CSP

---

**Navigation** : [<< Search-5 GeneticAlgorithms](Search-5-GeneticAlgorithms.ipynb) | [Index](../README.md) | [Search-7 CSP-Consistency >>](Search-7-CSP-Consistency.ipynb)